<h2>Phase 2: Creating the Interaction Pairs</h2>

After successfully loading all data from the 1999 to 2018 NHANES cycles into the MyNHANES system, we progress to the next phase which involves compiling a comprehensive list of fields related to exposure factors. We will utilize the detailed descriptions of these NHANES fields to operate the IGEM Search Engine, facilitating the retrieval of relevant TERMS and enabling thorough consultation of the existing relationships within our knowledge base.

- STEP 00_00 - Extract Fields List from MyNHANES
- STEP 01_00 - Search IGEM Terms from NHANES Fields Description
- STEP 01_01 - Identify Exposure Factor Fields
- STEP 01_02 - Generate Parameters File to setting the filter to GE.db database
- STEP 01_03 - Obtaining the Terms of Relationship
- STEP 01_04 - Filter Interactions
- STEP 01_05 - Link the IGEM Terms to NHANES Field ID
- STEP 01_06 - Isolate the Unique Fields
- STEP 02_00 - Select query in MyNhanes

In [1]:
import pandas as pd
from pathlib import Path
import glob
import os

In [2]:
path = Path().resolve()
path_data = path / 'data'

In [5]:
"""
Add commands to MyNHANES and IGEM
"""

'\nAdd commands to MyNHANES and IGEM\n'

In [3]:
df_terms = pd.read_csv(
    str(path_data) + "/step_01_03_TermsRelationship.csv"
    )
ls_terms = pd.read_excel(
    (str(path_data) + "/step_01_01_Exposes_Identification.xlsx"),
    sheet_name="Exposes_Terms_Uniques",
    header=None,
    names=['term']
    )['term'].tolist()

In [22]:
df_terms_filtered = df_terms[
    df_terms['term_1'].isin(ls_terms) & df_terms['term_2'].isin(ls_terms)
    ]
df_terms_filtered.drop(
    columns=[
        'datasource',
        'connector',
        'qtd_links'
        ],
    inplace=True
    )

df_terms_filtered.drop_duplicates(inplace=True)

df_terms_filtered.to_csv(
    str(path_data) + "/step_01_04_TermsRelationship_filtered.csv",
    index=False
)

/var/folders/06/f05rc_9x21nfs2142bf0fmcr0000gn/T/ipykernel_47655/826482400.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_terms_filtered.drop(
/var/folders/06/f05rc_9x21nfs2142bf0fmcr0000gn/T/ipykernel_47655/826482400.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_terms_filtered.drop_duplicates(inplace=True)


<h4>STEP 01_05 - Link the IGEM Terms to NHANES Field ID</h4>

In [5]:
df_NHAMES_fields = pd.read_excel(
    str(path_data) + "/step_00_00_MyNHANES_fields_list.xlsx",
    sheet_name="fields_unique"
    )
df_NHANES_terms = pd.read_csv(
    str(path_data) + "/step_01_00_word_to_term.csv"
    )

In [19]:
# Normalizando os campos de string para comparação
df_NHAMES_fields['field_description'] = df_NHAMES_fields['field_description'].str.lower().str.strip()
df_NHANES_terms['string'] = df_NHANES_terms['string'].str.lower().str.strip()

# Concatenando os DataFrames com base na coluna 'string' de df_NHANES_terms e 'field_description' de df_NHANES_fields
df_NHANES_fields_terms = pd.merge(df_NHANES_terms, df_NHAMES_fields, left_on='string', right_on='field_description', how='left')

In [21]:
# Eliminando as colunas 'datasource', 'connector' e 'qtd_links'
df_NHANES_fields_terms.drop(columns=['row', 'string', 'word', 'term_id', 'term_descr', 'qtd_terms', 'qtd_loops', 'time'], inplace=True)
# Eliminando registros duplicados
df_NHANES_fields_terms.drop_duplicates(inplace=True)
df_NHANES_fields_terms.dropna(subset=['term'], inplace=True)

In [22]:
df_NHANES_fields_terms

,term,field_name,field_description
0,path:r-hsa-168298,SDDSRVYR,data release cycle
1,gene:40162,SDDSRVYR,data release cycle
8,gene:31251,RIDEXMON,six month time period
32,meta:hmdb0302501,DMDHHSZA,# of children 5 years or younger in hh
44,error,INDHHIN2,annual household income
...,...,...,...
15931,meta:hmdb0002878,LBXWNO,"nitrate, water (ng/ml)"
15932,meta:hmdb0012238,LBDWIOLC,"iodide, water comment code"
15933,meta:hmdb0002111,LBDWIOLC,"iodide, water comment code"
15934,meta:hmdb0002111,LBXWP8,"perchlorate, water (ng/ml)"


In [34]:
df_interactions= df_terms_filtered.drop(columns=['term_group_1', 'term_category_1', 'word_1',  'description_1', 'term_group_2', 'term_category_2', 'word_2', 'description_2'])
df_interactions

,term_1,term_2
1073,chem:c000619812,dise:d006973
1080,chem:c000619812,dise:d007249
1098,chem:c000619812,dise:d008545
1140,chem:c000619812,dise:d015431
4785,chem:c010210,dise:d006973
...,...,...
317726,meta:hmdb0015532,meta:hmdb0302501
318671,meta:hmdb0000464,path:hsa05133
318677,meta:hmdb0000547,path:hsa05133
318679,meta:hmdb0000586,path:hsa05133


In [35]:
# Criando o novo DataFrame
new_rows = []

for _, model_row in df_interactions.iterrows():
    term_1 = model_row['term_1']
    term_2 = model_row['term_2']
    
    # Procurando correspondências para term_1
    term_1_matches = df_NHANES_fields_terms[df_NHANES_fields_terms['term'] == term_1]
    
    # Procurando correspondências para term_2
    term_2_matches = df_NHANES_fields_terms[df_NHANES_fields_terms['term'] == term_2]
    
    for _, term_1_row in term_1_matches.iterrows():
        for _, term_2_row in term_2_matches.iterrows():
            new_row = {
                'term_1': term_1,
                'field_name_1': term_1_row['field_name'],
                'field_description_1': term_1_row['field_description'],
                'term_2': term_2,
                'field_name_2': term_2_row['field_name'],
                'field_description_2': term_2_row['field_description']
            }
            new_rows.append(new_row)

df_models = pd.DataFrame(new_rows)

In [37]:
df_models.to_csv(
    str(path_data) + "/step_01_05_Models.csv",
    index=False
    )

In [39]:
df_models.columns


Index(['term_1', 'field_name_1', 'field_description_1', 'term_2',
       'field_name_2', 'field_description_2'],
      dtype='object')

Fase 01_06 - Isolar os fields Unicos 

In [51]:
df_fields_1 = df_models.drop(columns=['term_1', 'term_2', 'field_name_2', 'field_description_2'])
df_fields_2 = df_models.drop(columns=['term_1', 'term_2', 'field_name_1', 'field_description_1'])
df_fields_1.rename(columns={'field_name_1': 'field_name', 'field_description_1': 'field_description'}, inplace=True)
df_fields_2.rename(columns={'field_name_2': 'field_name', 'field_description_2': 'field_description'}, inplace=True)
df_fields = pd.concat([df_fields_1, df_fields_2], ignore_index=True)
df_fields.drop_duplicates(inplace=True)
df_fields.to_csv(
    str(path_data) + "/step_01_06_Fields.csv",
    index=False
    )

Step_02: Merge all fields from MYNHANES

In [19]:
folder_path = path_data / "Fields_by_Cycle"
csv_files = glob.glob(os.path.join(str(folder_path), "*.csv"))

['/Users/andrerico/Works/Projects/pbs_igem/data/Fields_by_Cycle/IGEM_ExE_2007-2008.csv',
 '/Users/andrerico/Works/Projects/pbs_igem/data/Fields_by_Cycle/IGEM_ExE_2009-2010.csv',
 '/Users/andrerico/Works/Projects/pbs_igem/data/Fields_by_Cycle/IGEM_ExE_2015-2016.csv',
 '/Users/andrerico/Works/Projects/pbs_igem/data/Fields_by_Cycle/IGEM_ExE_2003-2004.csv',
 '/Users/andrerico/Works/Projects/pbs_igem/data/Fields_by_Cycle/IGEM_ExE_2005-2006.csv',
 '/Users/andrerico/Works/Projects/pbs_igem/data/Fields_by_Cycle/IGEM_ExE_2013-2014.csv',
 '/Users/andrerico/Works/Projects/pbs_igem/data/Fields_by_Cycle/IGEM_ExE_1999-2000.csv',
 '/Users/andrerico/Works/Projects/pbs_igem/data/Fields_by_Cycle/IGEM_ExE_2017-2018.csv',
 '/Users/andrerico/Works/Projects/pbs_igem/data/Fields_by_Cycle/IGEM_ExE_2001-2002.csv',
 '/Users/andrerico/Works/Projects/pbs_igem/data/Fields_by_Cycle/IGEM_ExE_2011-2012.csv']

In [20]:
folder_path = path_data / "Fields_by_Cycle"
csv_files = glob.glob(os.path.join(str(folder_path), "*.csv"))

dataframes = []

for file in csv_files:
    df = pd.read_csv(file)
    dataframes.append(df)

combined_df = pd.concat(dataframes, ignore_index=True)

combined_df.to_csv("combined_file.csv", index=False)

In [22]:
combined_df.to_csv(
    str(path_data) + "/step_02_00_all_data.csv",
    index=False
    )